## Import Statements

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

from prophet import Prophet
import xgboost


## Getting Data

## Prophet

In [ ]:
#Data Prep for Prophet
train = pd.DataFrame()
test = pd.DataFrame()

train_pro = train
train_pro["y"] = train_pro["sales_actuals_monthly__vehiclegroup01__orderintake"]
train_pro = train.drop("sales_actuals_monthly__vehiclegroup01__orderintake", axis = 1)

train_pro["ds"] = train_pro.index

# Convert the 'ds' column to datetime format
start_date = '2000-01-01'  # Adjust this date to your needs
train_pro['ds'] = pd.to_datetime(train_pro['ds'].apply(lambda x: pd.DateOffset(months=x) + pd.Timestamp(start_date)))

In [ ]:
m = Prophet()

# Fit the model with train set
m.fit(train_pro)

future = m.make_future_dataframe(periods=18, freq = "MS")

# Make predictions on the test set dates
forecast_pro = m.predict(future)

forecast_pro = forecast_pro.iloc[240:, :]

# Calculate errors
mse = mean_squared_error(test_pro['y'], forecast_pro['yhat'])
mae = mean_absolute_error(test_pro['y'], forecast_pro['yhat'])

print('Mean Squared Error:', mse)
print('Mean Absolute Error:', mae)

In [ ]:

# Create a new figure
plt.figure()

# Plot y_test
plt.plot(test_pro["y"] , label='Actual')

# Plot y_pred
plt.plot(forecast_pro["yhat"], label='Predicted')

# Add a legend
plt.legend()

# Show the plot
plt.show()

## XGBRegressor

In [ ]:
lag = 18

y_train_XGB = y_train.iloc[:-lag]
y_test_XGB = y_train.iloc[-lag:]

X_train_XGB = X_train.iloc[:-lag, :]
X_test_XGB = X_train.iloc[-lag:, :]

print(y_train_XGB.shape)
print(y_test_XGB.shape)
print(X_train_XGB.shape)
print(X_test_XGB.shape)

In [ ]:
# Train a XGBClassifier
model_3 = xgboost.XGBRegressor(use_label_encoder=False, eval_metric="mae").fit(X_train_XGB, y_train_XGB)

In [ ]:
y_test_XGB = y_test_XGB.reset_index()
y_test_XGB = y_test_XGB.drop("index", axis = 1)

In [ ]:
forecast_xgb = model_3.predict(X_test_XGB)

# Create a new figure
plt.figure()

# Plot y_test
plt.plot(y_test_XGB , label='Actual')

# Plot y_pred
plt.plot(forecast_xgb, label='Predicted')

# Add a legend
plt.legend()

# Show the plot
plt.show()